@Abdulrahman Alabrash 

https://github.com/alabrashJr/DCNN-Julia


In [3]:
using Pkg;Pkg.update(); for p in ("Embeddings","DataFrames","DataStructures","DataFrames","FileIO","LinearAlgebra","Knet","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using DataStructures,DataFrames,FileIO,Embeddings,LinearAlgebra,DataFrames
#using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, progress, sgd, param, param0, dropout, relu, Data,minibatch

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
 Installed Widgets ───────────── v0.6.1
 Installed ZipFile ───────────── v0.8.1
 Installed Images ────────────── v0.17.3
 Installed PyCall ────────────── v1.91.1
 Installed IJulia ────────────── v1.18.0
 Installed Plots ─────────────── v0.24.0
 Installed FileIO ────────────── v1.0.6
 Installed WebSockets ────────── v1.5.2
 Installed Ratios ────────────── v0.3.1
 Installed MacroTools ────────── v0.5.0
 Installed Compat ────────────── v2.1.0
 Installed DocStringExtensions ─ v0.7.0
 Installed CUDAapi ───────────── v0.6.3
 Installed TranscodingStreams ── v0.9.3
 Installed PlotUtils ─────────── v0.5.7
 Installed Tokenize ──────────── v0.5.3
 Installed LibCURL ───────────── v0.5.0
 Installed SoftGlobalScope ───── v1.0.10
 Installed Tables ────────────── v0.1.18
 Installed Mux 

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 539816466986 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Precompiling CategoricalArrays [324d7699-5711-5eae-9e2f-1d82baa6b597]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 539816466986 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Precompiling DataStreams [9a8bc11e-79be-5b39-94d7-1ccc349a1a85]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.1/CodecZlib/1TI30.ji for CodecZlib [944b1d66-785c-5afd-91f1-9de20f533193]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/ec2-user/.julia/com

Read data from saved file 
load("TREC_sib.jld2","datas")-> returns revs, W, W2, word_idx_map, vocab
* revs Dict{String,Any} with 5 entries:y,num_words,tree,text,split
    * y-> label of the questions 1-5 
    * num_words-> length of questions 
    * tree -> 46-element Array{Array,1}, each array contains data_sibling(5)+data_tree(5) =10
    * text -> the question text 
    * split -> type of tuple (training, test , div)



* W wordembedding generated by google2vec, size(10098×300)
* W2 word embedding generated by -0.25 , 0.25 size(10098×300)
* word_idx_map Dict{Any,Any} with 10097 entries , unique word id dictionary 
* vocab DefaultDict{Any,Any,Int64} with 10097 entries, vocab and reptation dictionary

In [11]:
labels=["abbreviation", "entity", "description", "location" ,"numeric"," "]
revs, W, W2, word_idx_map, vocab=load("Data/TREC_sib.jld2","datas"); 
word_idx_map["ROOT"] = 0;

In [14]:
#change W2 with prof's method 
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]
W2=Embed(300,length(vocab))

Embed(P(KnetArray{Float32,2}(10097,300)))

In [139]:
#Transforms sentence into a list of indices. Pad with zeroes.
function get_text_mat(t,word_idx_map;max_l=56,filter_h=5)
    #t the text of question
    x=[] # output matrix
    pad=filter_h -1 # padding number
    for i in collect(1:pad);push!(x,0);end #adding padding 
    words=split(t)
    #extract the unique id of words in the question text and adding it to the matrix 
    for w in words
        if w in keys(word_idx_map);push!(x,word_idx_map[w])
        else; @show w ;end
    end    

    while length(x)<max_l+2*pad    # accomplish 64 +1 size by adding zeros till finish 
            push!(x,0)
    end
    
    return  x
end

get_text_mat (generic function with 2 methods)

In [124]:
#Transforms sentence into a list of indices. Pad with zeroes. 
function get_tree_rep(r,word_idx_map)
# question 
#@show t=r["tree"] #the tree of question
    each_sent=deepcopy(r)# output matrix
    for (j, each_word) in enumerate(each_sent[1:end-1])
        #@show (j, each_word)
            for (l, each_field) in enumerate(each_word)
           # @show (l, each_field)
                if each_field in keys( word_idx_map)
                #@show j,l ;
                    each_sent[j]=Array{Any,1}(each_sent[j])
                     each_sent[j][l] = word_idx_map[each_field]
                elseif each_field == 0
                    continue
                else
                    @show each_field
                end
            end
    end       
    return each_sent;
end

get_tree_rep (generic function with 1 method)

In [134]:
ran=rand(1:length(revs));@show ran
print(revs[ran]["text"]); println("\t",length(split(revs[ran]["text"])))
println(permutedims(get_text_mat(revs[ran]["text"],word_idx_map;max_l=56) ))
for i in split(revs[ran]["text"]) ;println(i,"=",word_idx_map[i]) ;end

ran = 4461
What U.S. state boasts Leif Ericson Park ?	8
Any[0 0 0 0 6965 4384 9578 5068 288 9386 8878 1557 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
What=6965
U.S.=4384
state=9578
boasts=5068
Leif=288
Ericson=9386
Park=8878
?=1557


In [16]:
ran=1#rand(1:length(revs))
tree_ind=1
println(revs[ran]["tree"][tree_ind])
println(permutedims(get_tree_rep(revs[ran]["tree"],word_idx_map))[tree_ind])
for i in revs[ran]["tree"][tree_ind] ;println(i,"=",word_idx_map[i]) ;end

Any["ROOT", "ROOT", "ROOT", "did", "develop", "did", "develop", "How", "*START*", "ROOT"]


UndefVarError: UndefVarError: get_tree_rep not defined

In [314]:
function train_dev_test(revs)
    s1,s2,s3=[],[],[]
    t1,t2,t3=[],[],[]
    for rev in revs
    sent =get_text_mat(rev["text"], word_idx_map)   
    push!(sent,rev["y"])
    sent_tensor = get_tree_rep(rev["tree"], word_idx_map)
        
    if rev["split"]==1
            push!(s1,Array{Int}(sent))
            push!(t1,sent_tensor)
    elseif rev["split"]==2
            push!(s2,Array{Int}(sent))
            push!(t2,sent_tensor)
    elseif rev["split"]==3
            push!(s3,Array{Int}(sent))
            push!(t3,sent_tensor)
    end
end

    train = hcat([f1 for f1 in s1]...)
    test =hcat([f1 for f1 in s2]...)
    dev = hcat([f1 for f1 in s3]...)
    train_tensor = t1
    test_tensor = t2
    dev_tensor = t3
    return (train,test,dev),(train_tensor,test_tensor,dev_tensor)
end
dataset,datasetTensor=train_dev_test(revs);


dataset[1] --> train 65×5451 } text data
dataset[2]----> test 65×501  }
dataset[3] ----> dev 0x0     }

datasetTensor[1] ---> train_tensor 5451(46(10)))
datasetTensor[2] ---> test_tensor 501(46(10)))
datasetTensor[3] --->dev_tensor 0(0(0))

In [315]:
function getSen(vector)
#labels=["abbreviation","numeric",  "description", "human","location" ,"entity"]
t=Array{Int}(vector)
println(permutedims(t))
for i in t
    if i==1557;print("?\n y =",t[end]+1 );break;end
    if i==0;continue;end
    for (key,value) in word_idx_map
        if value==i; print(key," ");end
    end
end
end

getSen (generic function with 1 method)

In [316]:
#try on some vectors 
ran=rand(1:size(dataset[1],2))
getSen(dataset[1][:,ran])

[0 0 0 0 6965 1189 9009 9078 4689 9231 4560 9231 1241 1557 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4]
What is the largest county in size in Massachusetts ?
 y =5

In [317]:
y_train=[dataset[1][:,x][end] for x in 1:size(dataset[1],2)];
y_test=[dataset[2][:,x][end] for x in 1:size(dataset[2],2)];

In [318]:
dtrn=minibatch(dataset[1][1:end-1,:],y_train,160,shuffle=true) # 5451/170
dtst=minibatch(dataset[2][1:end-1,:],y_test,160)

Data{Tuple{Array{Int64,2},Array{Int64,1}}}([0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [1 5 … 1 2], 160, 501, false, 342, 1:501, false, (64, 501), (501,), Array{Int64,2}, Array{Int64,1})

In [319]:
summary.(Iterators.first(dtrn))

("64×160 Array{Int64,2}", "160-element Array{Int64,1}")